In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
!mkdir models_debug
!mkdir models_folded
def kfold_model_inproc(model_fold_storage, problem_type, model_type, X, y, Xtest):
    import os
    print("kfold", str(os.getpid()))
    import tensorflow as tf
    from keras import backend as K
    import gc
    from tqdm import tqdm
    import pickle

    import numpy as np
    split = 5
    global pred_dim
    global pred_dim_list
    pred_dim_list = [i if i>0 else X.shape[0] for i in problem_type.pred_dim()]
    pred_dim = tuple(i for i in pred_dim_list)
    global oof_pred
    oof_pred = np.zeros(pred_dim)
    global test_pred
    test_pred_dim_list = [i if i>0 else Xtest.shape[0] for i in problem_type.pred_dim()]
    test_pred_dim = tuple(i for i in test_pred_dim_list)
    test_pred = np.zeros((split,) + test_pred_dim)
    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=split, shuffle=True)
    i_fold=0
    all_train_idx = []
    all_valid_idx = []
    all_model_files = []
    global all_metrics
    all_metrics = {}
    global train_idx
    global valid_idx
    for train_idx, valid_idx in tqdm(kf.split(X, y.argmax(1))):
        all_train_idx.append(train_idx)
        all_valid_idx.append(valid_idx)
    nfolds = len(all_valid_idx)
    for i_fold in range(nfold):
        train_idx = all_train_idx[i_fold]
        valid_idx = all_valid_idx[i_fold]
        Xt = X.iloc[train_idx]
        yt = y[train_idx]
        Xv = X.iloc[valid_idx]
        yv = y[valid_idx]
        
        model = model_type.create()
        model_type.fit(model, Xt, yt, Xv, yv)
        
        global oof_df_fold
        oof_df_fold = model_type.predict(model, Xv)
        global metrics
        metrics = model_type.evaluate(model, Xv, yv)
        for key in metrics:
            if key in all_metrics:
                all_metrics[key]=np.hstack((all_metrics[key], metrics[key]))
            else: 
                all_metrics[key]=metrics[key]
        global test_df_fold
        test_df_fold = model_type.predict(model, Xtest)
        model_fold_storage.save_fold(model_type, X, y, valid_idx, train_idx, i_fold, oof_df_fold, test_df_fold, metrics)
        
        oof_pred[valid_idx, :] = np.reshape(oof_df_fold, oof_df_fold.shape)
        test_pred[i_fold]=test_df_fold
        model_file = model_type.save(i_fold, model)
        all_model_files.append(model_file)
        i_fold+=1
        K.clear_session()
        del model
        gc.collect()
        K.clear_session()
        gc.collect()
        K.clear_session()
    model_fold_storage.save_kfold(model_type, X, y, all_valid_idx, all_train_idx, all_model_files, oof_pred, test_pred, all_metrics)
    return oof_pred, test_pred, all_metrics

A subdirectory or file models_debug already exists.
A subdirectory or file models_folded already exists.


In [3]:
!mkdir models
class ModelFoldStorage:
    def save_fold(self, model_type, X, y, valid_idx, train_idx, i_fold, oof_df_fold, test_df_fold, metrics):
        foldData = {}
        foldData['X'], foldData['y'],foldData['valid_idx'],foldData['train_idx']=X,y,valid_idx,train_idx
        foldData['oof_df_fold'],foldData['i_fold'], foldData['metrics'], foldData['test_df_fold']=oof_df_fold,i_fold, metrics, test_df_fold
        with open(r"models/"+model_type.name()+"_"+str(i_fold)+".bin", "wb") as output_file:
            pickle.dump(foldData, output_file)
    def save_kfold(self, model_type, X, y, all_valid_idx, all_train_idx, all_model_files, oof_pred, test_pred, all_metrics):
        modelTypeData = {}
        modelTypeData['X'], modelTypeData['y'],modelTypeData['all_valid_idx'],modelTypeData['all_train_idx'], modelTypeData['all_model_files']=X,y,all_valid_idx,all_train_idx, all_model_files
        modelTypeData['model_type'], modelTypeData['oof_pred'],modelTypeData['all_metrics'], modelTypeData['test_pred']=model_type, oof_pred, all_metrics, test_pred
        with open(r"models_folded/"+model_type.name()+".bin", "wb") as output_file:
            pickle.dump(modelTypeData, output_file)

In [4]:
class MyProblem:
    def pred_dim(self):
        return [0]

In [5]:
class MyModel:
    def name(self):
        pass
    def create(self):
        pass
    def fit(self, model, Xt, yt, Xv, yv):
        pass
    def predict(self, model, Xt):
        pass
    def evaluate(self, model, x, y):
        pass
    def save(self, i, model):
        pass
    def load(self, i):
        pass

In [6]:
def DatasetMapFunction(input_ids, attn_masks, labels):
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
            }, labels
class MyBertModel(MyModel):
    def __init__(self, debug=False):
        self.debug = debug
    def create(self):
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text as text
        from transformers import TFBertModel
        model = TFBertModel.from_pretrained("bert-base-uncased")
        input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
        attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

        bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
        intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
        output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

        discourse_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
        discourse_model.summary()
        from tensorflow.keras.optimizers import Adam
        discourse_model.compile(optimizer=Adam(learning_rate=1e-5, decay=1e-6), 
                        loss='categorical_crossentropy', 
                        metrics=['accuracy'])
        return (discourse_model, self, model)
    def name(self):
        return "bert"
    def fit(self, model, Xt, yt, Xv, yv):
        X_input_ids, X_attn_masks, yt = self._transform(Xt, yt)
        import tensorflow as tf
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, yt))
        dataset = dataset.map(DatasetMapFunction)     # converting to required format for tensorflow dataset
        dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
        X_val_input_ids, X_val_attn_masks, yv = self._transform(Xv, yv)
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val_input_ids, X_val_attn_masks, yv))
        val_dataset = val_dataset.map(DatasetMapFunction)     # converting to required format for tensorflow dataset
        val_dataset = val_dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
        epochs = 5
        if self.debug:
            epochs=1
        model[1].history = model[0].fit(dataset,
            steps_per_epoch=200,
            validation_data=val_dataset,
            epochs=epochs)
    def predict(self, model, X):
        X_test_input_ids, X_test_attn_masks, _y = self._transform(X, None)
        labels = model[0].predict([X_test_input_ids, X_test_attn_masks])
        return labels
    def evaluate(self, model, x, y):
        import tensorflow as tf
        X_input_ids, X_attn_masks, y = self._transform(x,y)
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, y))
        dataset = dataset.map(DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)
        return model[0].evaluate(dataset, return_dict=True)
    def _encode_data(self, df, ids, masks, tokenizer):
        from tqdm.auto import tqdm
        for i, text in tqdm(enumerate(df['text'])):
            tokenized_text = tokenizer.encode_plus(
                text,
                max_length=256, 
                truncation=True, 
                padding='max_length', 
                add_special_tokens=True,
                return_tensors='tf'
            )
            ids[i, :] = tokenized_text.input_ids
            masks[i, :] = tokenized_text.attention_mask
        return ids, masks
    
    def _transform(self, X,y):
        if self.debug:
            X = X.head(1000)
            if y is not None:
                y = y[0:1000,:]
        from transformers import BertTokenizerFast
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        import numpy as np
        X_input_ids = np.zeros((len(X), 256))
        X_attn_masks = np.zeros((len(X), 256))
        from tqdm.auto import tqdm
        X_input_ids, X_attn_masks = self._encode_data(X, X_input_ids, X_attn_masks, tokenizer)
        return X_input_ids, X_attn_masks, y

In [7]:
class FeedbackProblem(MyProblem):
    def pred_dim(self):
        return [0,3]

In [8]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

['.gitattributes', 'LICENSE', 'README.md', 'config.json', 'flax_model.msgpack', 'pytorch_model.bin', 'rust_model.ot', 'tf_model.h5', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


In [9]:
df_train = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
df_test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")

In [10]:
df_train["text"] = df_train["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/train/{x}.txt').read())
df_test["text"] = df_test["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/test/{x}.txt').read())

In [11]:
pd.set_option('display.max_colwidth', 2550)
df_train.head(2)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no life on Mars that we have descovered yet. If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.\n\nPeople thought that the face was formed by alieans because they thought that there was life on Mars. though some say that life on Mars does exist, I think that there is no life on Mars.\n\nIt says in paragraph 7, on April 5, 1998, Mars Global Surveyor flew over Cydonia for the first time. Michael Malin took a picture of Mars with his Orbiter Camera, that the face was a natural landform. Everyone who thought it was made by alieans even though it wasn't, was not satisfied. I think they were not satisfied because they have thought since 1976 that it was really formed by alieans.\n\nThough people were not satified about how the landform was a natural landform, in all, we new that alieans did not form the face. I would like to know how the landform was formed. we know now that life on Mars doesn't exist."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no life on Mars that we have descovered yet. If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.\n\nPeople thought that the face was formed by alieans because they thought that there was life on Mars. though some say that life on Mars does exist, I think that there is no life on Mars.\n\nIt says in paragraph 7, on April 5, 1998, Mars Global Surveyor flew over Cydonia for the first time. Michael Malin took a picture of Mars with his Orbiter Camera, that the face was a natural landform. Everyone who thought it was made by alieans even though it wasn't, was not satisfied. I t

In [12]:
df_test.head(2)

,discourse_id,essay_id,discourse_text,discourse_type,text
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself.,Lead,"Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. Seeking multiple opinions can help a person make a better choice because it can decrease stress levels, a great chance to learn something new, can be very helpful and beneficial.\n\nTo begin with, Seeking information from more than one person can decrease stress levels. When making a decision there is a chance that you can become very stressed and overwhelmed. Being stressed can cause a person to give up on what they are doing. Having multiple opinions from people can make choosing an option less difficult and more easier. The information that you receive from others may have a special meaning in the future. Other peoples opinion can make a person feel confident in making the right choice.\n\nWhen you ask other people for their opinion you can also learn many new things. Everyone is different and may have more experience than you. Seeking other peoples opinion can cause you to gain a new skill or lesson. For example, someones advice could teach you how to do something the correct way. Many people are very different and have experience different things in life. Seeking advice from others can teach you a lot. We all learn from our mistakes in life, by sharing your past experiences you may prevent someone else from making the same mistake.\n\nSeeking others opinion can be very helpful and beneficial. Taking other peoples advice and doing what they say may lead to a great outcome. When you receive other opinions whether they are good or bad you will be able to look at them from a different point of view. For example, When a group of people give you advice on yourself, you then know how they look at you as a person. Everyone looks at certain things from a different prospective. How someone else looks at it may be how its supposed to be viewed in life.\n\nIn conclusion, You can learn from others experiences by seeking the advice that someone give you. Making decisions can cause stress on a person. Relating to someones advice may higher your self-esteem because you no longer feel left out. Ask others for advice when making a difficult decision. When you seek others opinion more opportunities are available fro yourself."
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person make a better choice,Position,"Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. Seeking multiple opinions can help a person make a better choice because it can decrease stress levels, a great chance to learn something new, can be very helpful and beneficial.\n\nTo begin with, Seeking information from more than one person can decrease stress levels. When making a decision there is a chance that you can become very stressed and overwhelmed. Being stressed can cause a person to give up on what they are doing. Having multiple opinions from people can make choosing an option less difficult and more easier. The information that you receive from others may have a special meaning in the future. Other peoples opinion can make a person feel confident in making the right choice.\n\nWhen you ask other people for their opinion you can also learn many new things. Everyone is different and may have more experience

In [13]:
effectiveness_map = {"Ineffective":0, "Adequate":1,"Effective":2}
df_train["target"] = df_train["discourse_effectiveness"].map(effectiveness_map)

In [14]:
df_train['text']  = df_train['discourse_type'] + tokenizer.sep_token + df_train['text']
df_test['text']  = df_train['discourse_type'] + tokenizer.sep_token + df_train['discourse_text']

In [15]:
import numpy as np
labels = np.zeros((len(df_train), 3))
labels[np.arange(len(df_train)), df_train['target'].values] = 1
labels

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [16]:
problem_type = FeedbackProblem()

In [17]:
model_type = MyBertModel(debug=False)

In [18]:
kfold_model_inproc(problem_type, model_type, df_train, labels, df_test)

TypeError: kfold_model_inproc() missing 1 required positional argument: 'Xtest'

In [ ]:
metrics

In [ ]:
all_metrics